In [ ]:
MK                 = None
save_perc          = None
Density_ud         = None
Density_od         = None
times_we_tried_max = None
file_path_Lvl      = None
file_path_MK       = None

In [ ]:
%run ./7___Finder___Functions.ipynb

---
---
---

### Load dictionaries

In [ ]:
with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)
size = grid_lvld.shape[0]

In [ ]:
with open(file_path_Lvl+"ud_lvl___"+str(Density_ud)+".pk", 'rb') as f: ud_lvl                = pkl.load(f)
with open(file_path_Lvl+"od_lvl___"+str(Density_od)+".pk", 'rb') as f: od_lvl                = pkl.load(f)

with open(file_path_Lvl+"dict_origins_isolated.pk",        'rb') as f: dict_origins_isolated = pkl.load(f)
with open(file_path_Lvl+"dict_origins_pairs.pk",           'rb') as f: dict_origins_pairs    = pkl.load(f)
lvl = len(dict_origins_isolated)

---

### Voids to be merged

In [ ]:
# Filled grid - with the indices of each void
fg = np.zeros((size, size, size), dtype=int) - 1

In [ ]:
void_index_od = 0
found_merge = False

void_index = -1
for lvl_i in range(lvl):

    if len(dict_origins_isolated[lvl_i]) != 0:
        for isolated_i in dict_origins_isolated[lvl_i]:
            void_index += 1
            fg[tuple(isolated_i)] = void_index
            if (not found_merge) and (od_lvl < lvl_i):
                void_index_od = void_index
                found_merge = True

    if MK == "MK2":
        if len(dict_origins_pairs[lvl_i]) != 0:
            for pairs_i in dict_origins_pairs[lvl_i]:
                void_index += 1
                for pairs_ij in pairs_i:
                    fg[tuple(pairs_ij)] = void_index
                if (not found_merge) and (od_lvl < lvl_i):
                    void_index_od = void_index
                    found_merge = True

no_voids = np.max(fg)+1

In [ ]:
with open(file_path_MK+"fg___0.pk", 'wb') as f: pkl.dump(fg, f)

---

In [ ]:
ratio_perc = 100
lvl_perc   = lvl // ratio_perc

# We found this to be a good limit given our RAM.
max_lvls_per_list = round(lvl / 100 * (512/size)**3)
no_lists = lvl // max_lvls_per_list + (1 if (lvl % max_lvls_per_list)!=0 else 0)

In [ ]:
ran_out_of_ods = False

for i in range(no_lists):
    
    with open(file_path_Lvl+"dict_cells_isolated/"+str(i)+".pk", 'rb') as f: dict_cells_isolated = pkl.load(f)
    with open(file_path_Lvl+"dict_cells_pairs/"+str(i)   +".pk", 'rb') as f: dict_cells_pairs    = pkl.load(f)
    
    start_lvl = max_lvls_per_list*i
    end_lvl   = np.min([max_lvls_per_list*(i+1), lvl])
    for current_lvl in range(start_lvl, end_lvl):
        
        ud = False; od = False
        if current_lvl < ud_lvl:
            ud = True

        # Also, if we ran out of od's, no use in checkign for od connections or expecting them to save leftover pair cells.
        if not ran_out_of_ods:
            if current_lvl > od_lvl:
                if np.max(fg) >= void_index_od:
                    od = True

        # List of all neighbor values of each od void... where we limit ourselves to 1 for practical purposes (more than
        #     2 we use them as pairs).
        #   -1 - no neighbor / already solved this one
        # >= 0 - the index of the one void
        #   -2 - multiple neighbors
        od_connections_all_val = np.array([-1 for _ in range(no_voids-void_index_od+1)])
        
        # First, we remove all isolated origins from the level isolated dict.
        for i0 in dict_origins_isolated[current_lvl]: dict_cells_isolated[current_lvl].remove(i0)

        # MK1: remove pair cells
        # Our separation of isolated and pair origins is one of the trademarks of MK2. So is the
        #     non-random peeling process. Thus, in MK1 we add all the pairs of cells from each level
        #     into the isolated dict and we empty the pair origins dict.
        if MK == "MK1":
            for pairs_i in dict_cells_pairs[current_lvl]:
                for pairs_ij in pairs_i:
                    dict_cells_isolated[current_lvl].append(pairs_ij)
            dict_cells_pairs[  current_lvl] = []
            dict_origins_pairs[current_lvl] = []

        # For MK2, we remove all pair origins from the level dict.
        else:
            for pairs_i in dict_origins_pairs[current_lvl]:
                dict_cells_pairs[current_lvl] = [coords for coords in dict_cells_pairs[current_lvl] 
                                                 if set(map(tuple, coords)) != set(map(tuple, pairs_i))]
            
            # If we're before our under density level, all cells are distributed at random, since
            #     any difference the peeling process could make matters only in the walls, but now
            #     there are no walls since the voids merge.
            if ud:
                for i0 in dict_cells_pairs[current_lvl]:
                    for j0 in i0: dict_cells_isolated[current_lvl].append(j0)
                dict_cells_pairs[current_lvl] = []




        # RUN THE FINDER
        
        # Before we start, how do we deal with potholes?
        # A pothole is an od void that essentially we need to consider as a pothole that we need to fill up to the 
        #     current level and then let non-od voids fill it in as they touch it.
        # - If one void touches it in a level, then the whole od void becomes part of it.
        # - If multiple voids touch it in a level, then we need to consider that void as a pothole filled up to that
        #     level and that now we run loop_pairs on it.

        # BUT WE CAN'T KNOW A PRIORI IF AN OD VOID ONLY TOUCHES ONE VOID (by the end of the level).
        # This statement is the foundation for the following procedure in managing od's.

        # To achieve, this, we must fist run the whole level (isolated and pairs cells) and see which od voids are connected.
        # Of course, if two od voids neighbor each-other, they must first be merged (and all the connections of the later)
        #     now become part of the former's.

        # Sure, ideally, it would be best if we would find a connection within a pair and then start going into the od with the
        #     regular void. Otherwise, we give the other voids that have a longer path through this pair a head-start: it's like
        #     they have an fair race through the path, but it doesn't matter which one reached the od first, they all start
        #     chewing through it at the same time.
        # This is true... but the effect is generally small and the times this occurs are rare at most... and implementing an
        #     ideal code would require to do the isolated ones like above, then run all pairs for one peel, stop, check for 
        #     connections, then merge all isolated and/or pair cells to the (possibly multiple) od('s), then repeat this 
        #     stop-and-go for every peel (which is now for all pairs at once). All this would add so much processing time for
        #     little to no actual benefit.
        # Plus, this is all a matter of interpretation: what if, instead, we considered that we fill-in the od's based on the
        #     steepness and distance from a regular's connection to it and its center... like a race to the bottom: it is 
        #     subjective (or at least a different type of paper needs to study this beforehand, but it'd most likely be a waste
        #     of everyone's time).

        
        # We begin with the isolated cells.
        isolated_i = np.array(dict_cells_isolated[current_lvl])
        if len(isolated_i) != 0: loop_isolated(fg, isolated_i, od_connections_all_val, ud, od, void_index_od, MK, size)
                

        # The pair cells are a bit more tricky....
        # First of all, if we are before the od's or we ran out of them, we can just run the regular loop_pair.
        if not ud and MK == "MK2":
            pairs = dict_cells_pairs[current_lvl]
            if len(pairs) != 0:
                for pairs_i in pairs:
                    pairs_i = np.array(pairs_i)
    
                    # For starters, it would be a great time-saver to find-out if a pair only touches a single void... then we can
                    #     just attribute the whole pair to it.
                    exterior_outline_vals = check_neighbors_cube(fg, pairs_i, size, stop_after_2=False)
    
                    
                    # If it touches no voids, it is isolated... becomes a wall.
                    if   len(exterior_outline_vals) == 0:
                        for [i,j,k] in pairs_i: fg[i][j][k] = -2
    
                    # If it only touches one single (regular or od) void, it is part of that void.
                    elif len(exterior_outline_vals) == 1:
                        eov0 = exterior_outline_vals[0]
                        for [i,j,k] in pairs_i: fg[i][j][k] = eov0
    
    
                    # If it touches multiple voids, then it's time for loop_pairs.
                    else:
                        
                        if not od:
                            loop_pairs(fg, pairs_i, od_connections_all_val, size, False, void_index_od, times_we_tried_max=times_we_tried_max)
                        
                        else:
                            # If we have od's, then it gets tricky...
                            # We perform the same counting of neighbors as above, but now we separate them into regualr and ods.
                            # There are many such cases and below we list them all.
        
        
                            
                            # Get all pair's neighbors and their values.
                            exterior_outline_vals_regular = [_ for _ in exterior_outline_vals if _ <  void_index_od]
                            exterior_outline_vals_od      = [_ for _ in exterior_outline_vals if _ >= void_index_od]

                            
                            # If it touches multiple regular voids but no od ones, we run loop_pairs excluding the od hail Mary.
                            if   len(exterior_outline_vals_regular) >= 2 and len(exterior_outline_vals_od) == 0:
                                loop_pairs(fg, pairs_i, od_connections_all_val, size, False, void_index_od, times_we_tried_max=times_we_tried_max)
            
                            # If it touches multiple od voids but no regular ones, we simply add the pair to the first one (not that
                            #     it matters) and merge the rest.
                            elif len(exterior_outline_vals_regular) == 0 and len(exterior_outline_vals_od) >= 2:
    
                                eov0 = exterior_outline_vals_od[0]
                                for [i,j,k] in pairs_i: fg[i][j][k] = eov0
    
                                for eov in exterior_outline_vals_od[1:]:
                                    fg[fg==eov] = eov0
                                    odcav0 = od_connections_all_val[eov0-void_index_od]
                                    odcav  = od_connections_all_val[eov -void_index_od]
                                    od_connections_all_val[eov-void_index_od] = -1
                                    if odcav0 != -2:
                                        if   odcav == -2:
                                            od_connections_all_val[eov0-void_index_od] = -2
                                        elif odcav >=  0:
                                            if odcav0 == -1:
                                                od_connections_all_val[eov0-void_index_od] = odcav
                                            else:
                                                od_connections_all_val[eov0-void_index_od] = -2
    
                            else:
                                # But if it touches a mix of regular and od voids... we need to be careful.
                                # We run loop_pairs essentially excluding all od voids (as if they were -1's) and only if we are left with
                                #     some cells unresolved before the try_it_all_again process, then we look for connections with these od's.
                                # The reason we wait so long is that it makes no difference if we include these od's or not in terms of the 
                                #     spread within the pair, with little effect on the one within the od's (see dicussion above, at the start
                                #     of the loop_pair section), but it saves us lots of computational time.
                                loop_pairs(fg, pairs_i, od_connections_all_val, size, True, void_index_od, times_we_tried_max=times_we_tried_max)

        if od:
            # Now that we have all od's that neighbor one or multiple regulars, we can easily sort them out.
            for i0 in range(no_voids-void_index_od+1):
                index0 = void_index_od+i0
                
                if   od_connections_all_val[i0] >=  0:
                    # If they touch one void in this level, they become part of it.
                    fg[fg==index0] = od_connections_all_val[i0]
                    
                elif od_connections_all_val[i0] == -2:
                    # If they touch multiple, then we run loop_pair.
                    pairs_i = np.argwhere(fg==index0)
                    if MK == "MK2": loop_pairs(fg, pairs_i, od_connections_all_val, size, False, void_index_od, times_we_tried_max=1)
                    else:           loop_isolated_leftovers(fg, pairs_i, size)
                    od_connections_all_val[i0] = -1

        # If we want to save every 1%...
        if save_perc and ((current_lvl+1) % lvl_perc == 0) and (((current_lvl+1) // lvl_perc != ratio_perc) or (current_lvl+1 == lvl)):
            with open(file_path_MK+"fg___"+str(int((current_lvl+1) // lvl_perc * 100 / ratio_perc))+".pk", 'wb') as f: pkl.dump(fg, f)

---

### Thin the walls

In [ ]:
walls_removed = thin_walls(fg, size)

In [ ]:
with open(file_path_MK+"walls_removed.pk", 'wb') as f: pkl.dump(walls_removed, f)

### Leftover od's: turn into pairs

In [ ]:
found_leftovers = False
while np.max(fg) >= void_index_od:
    found_leftovers = True
    leftover_pairs_i = np.argwhere(fg==np.max(fg))
    loop_pairs(fg, leftover_pairs_i, od_connections_all_val, size, False, void_index_od, times_we_tried_max=1)

### Thin the walls again if we had some leftover od's

In [ ]:
if found_leftovers: walls_removed = thin_walls(fg, size)

In [ ]:
with open(file_path_MK+"fg___100_thin.pk", 'wb') as f: pkl.dump(fg, f)

---
---
---